# CH 3

#1. 미리 학습된 언어 모델

## Language Model
- 단어 시퀀스에 확률을 부여하는 모델. 단어 시퀀스를 입력받아 해당 시퀀스가 얼마나 그럴듯한지 확률을 출력하는 모델
- 전체 단어 시퀀스가 나타날 확률은 이전 단어들이 주어졌을 때 다음 단어가 등장할 확률의 연쇄와 같다
  - 언어 모델을 '이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델'이라고 정의하기도 함

1. 순방향 언어 모델(forward language model)
  - GPT, ELMo
2. 역방향 언어 모델(backward language model)
  - ELMo (ELMo는 순방향과 역방향 언어 모델 모두 활용하는 기법)

## Masked Language Model
- 학습 대상 문장에 빈칸을 만들어 놓고 해당 빈칸에 올 단어로 적절한 단어가 무엇일지 분류하는 과정으로 학습
- BERT

## Skip-gram Model
- 어떤 단어 앞뒤에 특정 범위를 정해 두고 이 범위 내에 어떤 단어들이 올지 분류하는 과정으로 학습
- Word2Vec

# 2. Transformer
- 2017년 구글이 제안한 시퀀스-투-시퀀스 모델

## 시퀀스-투-시퀀스
- 특정 속성을 지닌 시퀀스를 다른 속성의 시퀀스로 변환하는 작업
- 소스와 타깃의 길이가 달라도 해당 과제를 수행하는 데 문제가 없어야 함

## 인코더와 디코더
- 인코더 : 소스 시퀀스의 정보를 압축해 디코더로 보냄
- 디코더 : 인코더가 보내 준 소스 시퀀스 정보를 받아서 타깃 시퀀스를 생성

## 모델 학습과 인퍼런스
- 트랜스포머는 인코더와 디코더 입력이 주어졌을 때 정답에 해당하는 단어의 확률값을 높이는 방식으로 학습한다


## 트랜스포머 블록
- 인코더 블록
  1. 멀티 헤드 어텐션
  2. 피드 포워드 뉴럴 네트워크
  3. 잔차 연결 및 레이어 정규화
- 디코더 블록
  1. 마스크 멀티 헤드 어텐션
  2. 멀티 헤드 어텐션
  3. 피드 포워드 뉴럴 네트워크
  4. 잔차 연결 및 레이어 정규화

### 셀프 어텐션이란?
- 어텐션 : 시퀀스 입력에 수행하는 기계학습 방법의 일종. 시퀀스 요소 가운데 중요한 요소에 집중하고 그렇지 않은 요소는 무시해 태스크 수행 성능을 끌어올림
- 셀프 어텐션
  - 입력 시퀀스 가운데 태스크 수행에 의미 있는 요소들 위주로 정보를 추출 
  - 입력 시퀀스 전체 전체 단어들 사이를 연결한다
  - 쿼리,키,밸류가 서로 영향을 주고받으면서 문장의 의미를 계산
  - 블록(레이어) 수만큼 반복

In [ ]:
import torch
x = torch.tensor([
    [1.0,0.0,1.0,0.0],
    [0.0,2.0,0.0,2.0],
    [1.0,1.0,1.0,1.0]])
w_query = torch.tensor([
    [1.0,0.0,1.0],
    [1.0,0.0,0.0],
    [0.0,0.0,1.0],
    [0.0,1.0,1.0]
])
w_key = torch.tensor([
    [0.0,0.0,1.0],
    [1.0,1.0,0.0],
    [0.0,1.0,0.0],
    [1.0,1.0,0.0]
])
w_value = torch.tensor([
    [0.0,2.0,0.0],
    [0.0,3.0,0.0],
    [1.0,0.0,3.0],
    [1.0,1.0,0.0]
])

In [ ]:
keys = torch.matmul(x,w_key)
querys = torch.matmul(x,w_query)
values = torch.matmul(x,w_value)

In [ ]:
attn_scores = torch.matmul(querys, keys.T)
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [ ]:
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_probs = softmax(attn_scores/key_dim_sqrt, dim=-1)
attn_probs

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

In [ ]:
weighted_values = torch.matmul(attn_probs, values)
weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

### 멀티 헤드 어텐션
- 셀프 어텐션을 동시에 여러번 수행하는 것. 여러 헤드가 독자적으로 셀프 어텐션을 계산한다는 뜻

### 마스크 멀티 헤드 어텐션
- 타깃 언어의 단어 벡터 시퀀스를 계산 대상으로 한다
- 한국어를 영어로 번역하는 태스크를 수행하는 트랜스포머 모델이라면 여기서 계산되는 대상은 영어 단어 시퀀스
- 정답을 포함한 타깃 시퀀스의 미래 정보를 셀프 어텐션 계산에서 제외(마스킹)하게 된다

### 피드포워드 뉴럴 네트워크
- 피드포워드 뉴럴 네트워크의 입력은 현재 블록의 멀티 헤드 어텐션의 개별 출력 벡터
- 활성 함수로 ReLU 사용

In [ ]:
import torch
x = torch.tensor([2,1])
w1 = torch.tensor([[3,2,-4],[2,-3,1]])
b1 =1 
w2 = torch.tensor([[-1,1],[1,2],[3,1]])
b2 = -1

In [ ]:
h_preact = torch.matmul(x,w1) + b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h,w2)+b2
y

tensor([-8, 12])

### 잔차 연결
- 블록이나 레이어 계산을 건너뛰는 경로를 하나 두는 것 { F(x)+x }

### 레이어 정규화
- 미니 배치의 인스턴스 별로 평균을 빼주고 표준편차로 나눠 정규활르 수행하는 기법
- 학습이 안정되고 그 속도가 빨라지는 효과

In [ ]:
import torch
input = torch.tensor([[1.0,2.0,3.0],[1.0,1.0,1.0]])
m = torch.nn.LayerNorm(input.shape[-1])
output = m(input)
output

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)

## 트랜스포머 모델의 학습 기법
1. 드롭아웃
  - 과적합을 방지하고자 뉴런의 일부를 확률적으로 0으로 대치하여 계산에서 제외
  - 학습 과정에만 적용하고 학습이 끝나고 나서 인퍼런스 과정에서는 사용 X
2. Adam Optimizer
  - 방향과 보폭을 적절하게 정해줌

# 3.GPT와 BERT 비교
- GPT : 트랜스포머에서 '디코더'만 사용 (멀티 헤드 어텐션 제외)
- BERT : 트랜스포머에서 '인코더'만 사용


#4. 단어/문장을 벡터로 변환하기


In [ ]:
!pip install ratsnlp

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'beomi/kcbert-base',
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [ ]:
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    'beomi/kcbert-base'
)
model = BertModel.from_pretrained(
    'beomi/kcbert-base',
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences = ['안녕하세요','하이!']
features = tokenizer(
    sentences,
    max_length = 10,
    padding = 'max_length',
    truncation=True
)

In [ ]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [ ]:
features = {k: torch.tensor(v) for k,v in features.items()}

In [ ]:
output = model(**features)

In [ ]:
output.last_hidden_state

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
output.pooler_output

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

- 이처럼 자연어를 벡터로 바꾼 결과를 Embedding 또는 Representation